# Confiabilidade de viga mista

## Importando bibliotecas

In [77]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm

## Definindo funções

In [ ]:
def calculo_destribuicao(media, desvpad) -> list:
    amostra: int = 1000
    medias: list = []

    for i in range(amostra):
        numero_aleatorio: float = np.random.rand()
        medias.append(norm.ppf(numero_aleatorio, media, desvpad))
        
    return medias
        
def calculo_destribuicao_areas(media, desvpad) -> list:
    amostra: int = 1000
    medias: list = []
    areas: list = []

    for i in range(amostra):
        numero_aleatorio: float = np.random.rand()
        medias.append(norm.ppf(numero_aleatorio, media, desvpad))
        areas.append(np.pi * (medias[i] / 2) ** 2)
        
    return medias, areas

## Espessura da laje

In [78]:
espessura_laje: float = 10
desvpad_espessura_laje: float = 0.06 * espessura_laje
amostra: int = 1000

espessuras: list = calculo_destribuicao(espessura_laje, desvpad_espessura_laje)

## Limite de escoamento do perfil de aço

In [79]:
limite_escoamento_perfil: float = 250 * 1.08
desvpad_escoamento_perfil: float = 0.08 * limite_escoamento_perfil

limites_escoamento_perfil: list = calculo_destribuicao(limite_escoamento_perfil, desvpad_escoamento_perfil)

## Limite de escoamento da armadura

In [80]:
limite_escoamento_armadura: float = 500 * 1.08
desvpad_escoamento_armadura: float = 0.08 * limite_escoamento_armadura

limites_escoamento_perfil: list = calculo_destribuicao(limite_escoamento_armadura, desvpad_escoamento_armadura)

## Resistência do concreto à compressão

In [81]:
resistencia_concreto: float = 20 * 1.17
desvpad_resistencia_concreto: float = 0.15 * resistencia_concreto

resistencias_concreto: list = calculo_destribuicao(resistencia_concreto, desvpad_resistencia_concreto)

## Diamêtro do rebite

In [83]:
diametro_rebite: float = 1.59
desvpad_diametro_rebite: float = 0.01 * diametro_rebite

diametros_rebites, areas_rebites = calculo_destribuicao_areas(diametro_rebite, desvpad_diametro_rebite)


## Resistência do rebite

In [84]:
resistencia_rebite: float = 1.59
desvpad_resistencia_rebite: float = 0.01 * resistencia_rebite

resistencias_rebite: list = calculo_destribuicao(resistencia_rebite, desvpad_resistencia_rebite)